In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, ArrayType

In [0]:
# ================================================================
# Parâmetro recebido via Databricks Job
# ================================================================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("schema_out", "")
dbutils.widgets.text("table", "")
dbutils.widgets.text("table_out", "")
dbutils.widgets.text("data_ref_carga", "")
catalog         = dbutils.widgets.get("catalog")
schema          = dbutils.widgets.get("schema")
schema_out      = dbutils.widgets.get("schema_out")
table           = dbutils.widgets.get("table")
table_out       = dbutils.widgets.get("table_out")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

In [0]:
# Ler a tabela bronze
df_bronze = spark.table(f"{catalog}.{schema}.{table}")

In [0]:
# ============================================================
# 2️⃣ Helper: extrair o primeiro bounding box não-nulo
# ============================================================
def get_geographic_bbox(field):
    """
    Retorna o campo dentro de identificationInfo.MD_DataIdentification.extent
    que contém EX_GeographicBoundingBox válido.
    """
    return F.expr(f"""
        filter(
            transform(
                {field},
                x -> x.EX_Extent.geographicElement.EX_GeographicBoundingBox
            ),
            y -> y is not null
        )[0]
    """)

In [0]:
# ============================================================
# 3️⃣ Criar coluna bbox antes de fazer o select final
# ============================================================
df_temp = df_bronze.withColumn(
    "bbox",
    get_geographic_bbox("identificationInfo.MD_DataIdentification.extent")
)

In [0]:
# ============================================================
# 4️⃣ Selecionar campos finais da camada Silver
# ============================================================
df_silver = df_temp.select(
    "data_ref_carga",

    # 🧾 Campos principais
    F.col("identificationInfo.MD_DataIdentification.abstract.CharacterString").alias("abstract"),
    F.col("identificationInfo.MD_DataIdentification.purpose.CharacterString").alias("purpose"),
    F.col("identificationInfo.MD_DataIdentification.citation.CI_Citation.title.CharacterString").alias("title"),
    F.col("identificationInfo.MD_DataIdentification.citation.CI_Citation.date.CI_Date.date.Date").alias("publication_date"),

    # 🌍 Campos geográficos extraídos do bbox
    F.col("bbox.eastBoundLongitude.Decimal").alias("eastBoundLongitude"),
    F.col("bbox.westBoundLongitude.Decimal").alias("westBoundLongitude"),
    F.col("bbox.northBoundLatitude.Decimal").alias("northBoundLatitude"),
    F.col("bbox.southBoundLatitude.Decimal").alias("southBoundLatitude"),

    # 🖼️ Ilustração
    F.col("identificationInfo.MD_DataIdentification.graphicOverview.MD_BrowseGraphic.fileName.CharacterString").alias("graphic_file"),
    F.col("identificationInfo.MD_DataIdentification.graphicOverview.MD_BrowseGraphic.fileDescription.CharacterString").alias("graphic_description"),

    # ⚖️ Licença / restrições
    F.col("identificationInfo.MD_DataIdentification.resourceConstraints.MD_LegalConstraints.otherConstraints.CharacterString").alias("license_info"),

    # 👤 E-mails de contato
    F.expr("""
        transform(
            identificationInfo.MD_DataIdentification.pointOfContact,
            x -> x.CI_ResponsibleParty.contactInfo.CI_Contact.address.CI_Address.electronicMailAddress.CharacterString
        )
    """).alias("emails"),

    # 🏷️ Palavras-chave
    F.expr("""
        flatten(
            transform(
                identificationInfo.MD_DataIdentification.descriptiveKeywords,
                x -> transform(x.MD_Keywords.keyword, y -> y.CharacterString)
            )
        )
    """).alias("keywords")
)

In [0]:
display(df_silver)

In [0]:
# Gravar tabela Silver Delta particionada por data_ref_carga
df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("data_ref_carga") \
    .saveAsTable(f"{catalog}.{schema_out}.{table_out}")